# 🍌🍎 Fruit Classifier — Uses `dataset/` with mapping `negatives -> none`

This end-to-end notebook loads images from your structure:
```
dataset/
  apples/  (desk, hand, hard, mixed, plain ... inside)
  bananas/ (...)
  oranges/ (...)
  negatives/ (...)
```
It maps **`negatives` to the class `none`**, builds `dataset_ready` (train/val/test),
trains a ResNet18 classifier, evaluates on test, and provides robust inference
on a custom image path (Windows OK).

Run top-to-bottom. Edit paths in the Config cell if needed.

## 1) Imports & Configuration

In [4]:
# --- Imports
import os, stat, shutil, time, random
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
from PIL import Image, ImageOps, ImageFile

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms as T

try:
    from tqdm import tqdm
except Exception:
    def tqdm(x, **k): return x

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Be resilient to slightly truncated jpgs
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

# --- Config (EDIT ME) ---
RAW_ROOT   = Path('dataset')          # << Your folder shown in VS Code
READY_ROOT = Path('dataset_ready')    # will be created
ARTIFACTS  = Path('artifacts'); ARTIFACTS.mkdir(exist_ok=True)

# Map your source folders to the *target* class names used for training
SRC_TO_DST = {
    'apples': 'apples',
    'bananas': 'bananas',
    'oranges': 'oranges',
    'negatives': 'none',   # <- map negatives to the class 'none'
}

# TARGET class list (order doesn't matter here; ImageFolder will set final order)
TARGET_CLASSES = sorted(set(SRC_TO_DST.values()))

# Training hyperparams
BASE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-3
WEIGHT_DECAY = 1e-4
USE_PRETRAINED = True
FREEZE_BACKBONE_EPOCHS = 0

print('RAW_ROOT   :', RAW_ROOT.resolve())
print('READY_ROOT :', READY_ROOT.resolve())
print('ARTIFACTS  :', ARTIFACTS.resolve())
print('Target classes:', TARGET_CLASSES)


Device: cuda
RAW_ROOT   : C:\Users\bilal\OneDrive\Projects\Object Detector\dataset
READY_ROOT : C:\Users\bilal\OneDrive\Projects\Object Detector\dataset_ready
ARTIFACTS  : C:\Users\bilal\OneDrive\Projects\Object Detector\artifacts
Target classes: ['apples', 'bananas', 'none', 'oranges']


## 2) Windows-safe delete and optional dataset health check

In [5]:
def safe_rmtree(path, retries=10, delay=0.3):
    path = Path(path)
    if not path.exists():
        return
    def onerror(func, p, exc_info):
        try:
            os.chmod(p, stat.S_IWRITE)
        except Exception:
            pass
        try:
            func(p)
        except PermissionError:
            raise
    for _ in range(retries):
        try:
            shutil.rmtree(path, onerror=onerror)
            return
        except PermissionError:
            time.sleep(delay)
    trash = path.with_name(path.name + f'._trash_{int(time.time())}')
    path.rename(trash)
    print(f'[safe_rmtree] Could not remove {path}. Renamed to {trash}. Delete it later.')

def dataset_health_check(root: Path, quarantine: Path):
    root, quarantine = Path(root), Path(quarantine)
    quarantine.mkdir(parents=True, exist_ok=True)
    total = bad = 0
    for p in root.rglob('*'):
        if p.suffix.lower() not in ('.jpg','.jpeg','.png','.bmp'):
            continue
        total += 1
        try:
            with Image.open(p) as im:
                im.verify()
            with Image.open(p) as im:
                _ = im.tobytes()
        except Exception as e:
            bad += 1
            dest = quarantine / p.name
            try:
                shutil.move(str(p), str(dest))
                print(f'[health_check] Moved bad image -> {dest}')
            except Exception as ee:
                print(f'[health_check] Could not move {p}: {ee}')
    print(f'[health_check] Scanned {total}, quarantined {bad}.')


## 3) Build `dataset_ready` with mapping (negatives -> none)

In [6]:
def dataset_ready_map(
    src_root: Path,
    dst_root: Path,
    src_to_dst: Dict[str, str],
    val_ratio: float = 0.15,
    test_ratio: float = 0.15,
    resize_to: int = 224,
    exts: Tuple[str, ...] = ('.jpg','.jpeg','.png','.bmp'),
):
    src_root, dst_root = Path(src_root), Path(dst_root)
    # Remove previous ready folder
    if dst_root.exists():
        print(f'[dataset_ready] Removing existing {dst_root}')
        safe_rmtree(dst_root)
    # Create destination class folders for each split
    dst_classes = sorted(set(src_to_dst.values()))
    for split in ['train','val','test']:
        for cls in dst_classes:
            (dst_root / split / cls).mkdir(parents=True, exist_ok=True)

    # Process each SOURCE class folder and place files into DEST class
    for src_cls, dst_cls in src_to_dst.items():
        src_dir = src_root / src_cls
        assert src_dir.exists(), f'Missing source folder: {src_dir}'
        files = sorted([p for p in src_dir.rglob('*') if p.suffix.lower() in exts])
        random.shuffle(files)
        n = len(files)
        n_test = int(n * test_ratio)
        n_val  = int(n * val_ratio)
        n_train = n - n_val - n_test
        splits = {
            'train': files[:n_train],
            'val'  : files[n_train:n_train+n_val],
            'test' : files[n_train+n_val:],
        }
        print(f'{src_cls} -> {dst_cls}: train={len(splits["train"])}, val={len(splits["val"])}, test={len(splits["test"])}')
        kept = skipped = 0
        for split, paths in splits.items():
            for i, src in enumerate(paths, 1):
                try:
                    with Image.open(src) as im:
                        im = ImageOps.exif_transpose(im).convert('RGB')
                        if resize_to:
                            im = im.resize((resize_to, resize_to), Image.BILINEAR)
                        dst = dst_root / split / dst_cls / f'{src.stem}_{i:05d}.jpg'
                        im.save(dst, quality=95, optimize=True)
                        with Image.open(dst) as chk:
                            chk.verify()
                    kept += 1
                except Exception:
                    skipped += 1
        print(f'{src_cls} -> {dst_cls}: kept={kept}, skipped={skipped}')
    print('[dataset_ready] Done ->', dst_root)


## 4) Prepare dataset (optional health check -> build `dataset_ready`)

In [7]:
# Optional health check
QUAR = RAW_ROOT.parent / 'quarantine_bad_images'
if RAW_ROOT.exists():
    dataset_health_check(RAW_ROOT, QUAR)
else:
    print('[warn] RAW_ROOT not found:', RAW_ROOT)

# Build ready dataset using mapping (negatives -> none)
dataset_ready_map(RAW_ROOT, READY_ROOT, SRC_TO_DST, resize_to=224)


[health_check] Scanned 2548, quarantined 0.
[dataset_ready] Removing existing dataset_ready
apples -> apples: train=430, val=91, test=91
apples -> apples: kept=612, skipped=0
bananas -> bananas: train=546, val=117, test=117
bananas -> bananas: kept=780, skipped=0
oranges -> oranges: train=449, val=96, test=96
oranges -> oranges: kept=641, skipped=0
negatives -> none: train=361, val=77, test=77
negatives -> none: kept=515, skipped=0
[dataset_ready] Done -> dataset_ready


## 5) Datasets & Dataloaders

In [8]:
train_transform = T.Compose([
    T.RandomResizedCrop(size=BASE_SIZE, scale=(0.7, 1.0)),
    T.RandomHorizontalFlip(),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
eval_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(BASE_SIZE),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_ds = datasets.ImageFolder(READY_ROOT/'train', transform=train_transform)
val_ds   = datasets.ImageFolder(READY_ROOT/'val',   transform=eval_transform)
test_ds  = datasets.ImageFolder(READY_ROOT/'test',  transform=eval_transform)

# Trust ImageFolder's class order
CLASS_ORDER = train_ds.classes
print('ImageFolder class order:', CLASS_ORDER)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0, pin_memory=torch.cuda.is_available())
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())

len(train_ds), len(val_ds), len(test_ds)


ImageFolder class order: ['apples', 'bananas', 'none', 'oranges']


(1786, 381, 381)

## 6) Build model (ResNet18)

In [9]:
num_classes = len(CLASS_ORDER)
if USE_PRETRAINED:
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
else:
    model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)
print(model.fc)


Linear(in_features=512, out_features=4, bias=True)


## 7) Train with validation and checkpoint

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max(EPOCHS, 1))

@torch.inference_mode()
def evaluate(loader):
    model.eval()
    total, correct, total_loss = 0, 0, 0.0
    for images, labels in tqdm(loader, leave=False):
        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels).item()
        total_loss += loss * images.size(0)
        preds = logits.argmax(1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
    return total_loss / max(total, 1), correct / max(total, 1)

def train_one_epoch():
    model.train()
    total, correct, total_loss = 0, 0, 0.0
    for images, labels in tqdm(train_loader, leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad(set_to_none=True)
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * images.size(0)
        preds = logits.argmax(1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
    return total_loss / max(total, 1), correct / max(total, 1)

best_val = 0.0
BEST_PATH = ARTIFACTS / 'best_model.pth'
for epoch in range(1, EPOCHS+1):
    if epoch == 1 and FREEZE_BACKBONE_EPOCHS > 0:
        for name, p in model.named_parameters():
            if not name.startswith('fc.'):
                p.requires_grad = False
    if epoch == FREEZE_BACKBONE_EPOCHS + 1:
        for p in model.parameters():
            p.requires_grad = True

    tr_loss, tr_acc = train_one_epoch()
    va_loss, va_acc = evaluate(val_loader)
    scheduler.step()

    print(f'Epoch {epoch:>2}/{EPOCHS} | Train loss {tr_loss:.4f} acc {tr_acc:.4f} | Val loss {va_loss:.4f} acc {va_acc:.4f}')
    if va_acc > best_val:
        best_val = va_acc
        torch.save(model.state_dict(), BEST_PATH)
        print('  -> saved new best to', BEST_PATH)
print('Best val acc:', best_val)


Epoch  1/10 | Train loss 0.3412 acc 0.8796 | Val loss 0.2036 acc 0.9291
  -> saved new best to artifacts\best_model.pth


Epoch  2/10 | Train loss 0.1709 acc 0.9451 | Val loss 0.4023 acc 0.8635


Epoch  3/10 | Train loss 0.1589 acc 0.9518 | Val loss 0.1452 acc 0.9580
  -> saved new best to artifacts\best_model.pth


Epoch  4/10 | Train loss 0.1027 acc 0.9681 | Val loss 0.1205 acc 0.9580


Epoch  5/10 | Train loss 0.0826 acc 0.9714 | Val loss 0.1228 acc 0.9554


Epoch  6/10 | Train loss 0.0576 acc 0.9804 | Val loss 0.0837 acc 0.9738
  -> saved new best to artifacts\best_model.pth


Epoch  7/10 | Train loss 0.0516 acc 0.9849 | Val loss 0.0492 acc 0.9816
  -> saved new best to artifacts\best_model.pth


Epoch  8/10 | Train loss 0.0419 acc 0.9854 | Val loss 0.0596 acc 0.9816


Epoch  9/10 | Train loss 0.0246 acc 0.9922 | Val loss 0.0489 acc 0.9816


Epoch 10/10 | Train loss 0.0231 acc 0.9899 | Val loss 0.0482 acc 0.9843
  -> saved new best to artifacts\best_model.pth
Best val acc: 0.984251968503937


## 8) Test evaluation (load best checkpoint)

In [11]:
BEST_PATH = ARTIFACTS / 'best_model.pth'
if BEST_PATH.exists():
    model.load_state_dict(torch.load(BEST_PATH, map_location=device))
    model.to(device).eval()
else:
    print('[warn] Best model not found, evaluating current weights.')

@torch.inference_mode()
def evaluate_with_confusion(loader, n_classes):
    cm = np.zeros((n_classes, n_classes), dtype=np.int64)
    total, correct, total_loss = 0, 0, 0.0
    for images, labels in tqdm(loader, leave=False):
        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels).item()
        total_loss += loss * images.size(0)
        preds = logits.argmax(1)
        for t, p in zip(labels.view(-1).cpu().numpy(), preds.view(-1).cpu().numpy()):
            cm[t, p] += 1
        correct += (preds == labels).sum().item()
        total += images.size(0)
    return total_loss / max(total,1), correct / max(total,1), cm

te_loss, te_acc, cm = evaluate_with_confusion(test_loader, len(CLASS_ORDER))
print(f'Test loss {te_loss:.4f} | Test acc {te_acc:.4f}')
print('Confusion matrix (rows=true, cols=pred):\n', cm)


Test loss 0.0406 | Test acc 0.9816
Confusion matrix (rows=true, cols=pred):
 [[ 89   0   2   0]
 [  0 116   1   0]
 [  0   2  75   0]
 [  0   2   0  94]]


## 9) Inference on a custom image (Windows path OK)

In [ ]:
@torch.inference_mode()
def _predict_tensor(x):
    model.eval()
    logits = model(x.to(device))
    probs = torch.softmax(logits, dim=1).squeeze(0).cpu().numpy()
    return int(probs.argmax()), probs

# Must match training eval transform
eval_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(BASE_SIZE),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def predict_image_fixed(path: str):
    img = Image.open(path).convert('RGB')
    x = eval_transform(img).unsqueeze(0)
    idx, probs = _predict_tensor(x)
    return CLASS_ORDER[idx], probs

def predict_image_multiscale_center(path: str, scales=(1.0, 0.85, 0.7)):
    img = Image.open(path).convert('RGB')
    w, h = img.size
    best = None
    for s in scales:
        side = int(min(w, h) * s)
        left, top = max(0, (w - side)//2), max(0, (h - side)//2)
        crop = img.crop((left, top, left + side, top + side))
        x = eval_transform(crop).unsqueeze(0)
        idx, probs = _predict_tensor(x)
        conf = float(probs[idx])
        if best is None or conf > best['conf']:
            best = {'idx': idx, 'probs': probs, 'conf': conf, 'box': (left, top, left + side, top + side)}
    return CLASS_ORDER[best['idx']], best['probs'], best['box']

# Set your test image path here (raw string for Windows backslashes)
EXAMPLE_PATH = r"C:\Users\bilal\OneDrive\Projects\Object Detector\samples\test_image.jpg"
print('EXAMPLE_PATH:', EXAMPLE_PATH)

labelA, probsA = predict_image_fixed(EXAMPLE_PATH)
print('Strict center-crop ->', labelA)
print('Probs:', {cls: float(probsA[i]) for i, cls in enumerate(CLASS_ORDER)})

labelB, probsB, box = predict_image_multiscale_center(EXAMPLE_PATH)
print('Multi-scale center ->', labelB, '| crop box:', box)
print('Probs:', {cls: float(probsB[i]) for i, cls in enumerate(CLASS_ORDER)})


EXAMPLE_PATH: C:\Users\bilal\OneDrive\Projects\Object Detector\Sample\test_image.jpg
Strict center-crop -> none
Probs: {'apples': 0.013024981133639812, 'bananas': 6.282403046498075e-05, 'none': 0.986713707447052, 'oranges': 0.00019841204630210996}
Multi-scale center -> none | crop box: (80, 0, 560, 480)
Probs: {'apples': 0.01567905955016613, 'bananas': 7.926952093839645e-05, 'none': 0.9840013980865479, 'oranges': 0.00024020539422053844}


In [14]:
print("Notebook class order:", CLASS_ORDER)

Notebook class order: ['apples', 'bananas', 'none', 'oranges']


## 10) Notes
- The mapping lets you keep your current folder names; `negatives` becomes the training class `none`.
- You can add more nested subfolders under each class; they are all included.
- If Windows locks `dataset_ready`, the safe remover will retry or rename and continue.